# 06. Trajectory Tracing for Training

Create a synthetic run and verify trace artifacts for training use.


In [ ]:
from __future__ import annotations

import json
import os
import math
import random
import statistics
from pathlib import Path


def find_project_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / 'README.md').exists() and (candidate / 'main_langgraph.py').exists():
            return candidate
    return start


PROJECT_ROOT = find_project_root(Path.cwd().resolve())
os.chdir(PROJECT_ROOT)
print('PROJECT_ROOT =', PROJECT_ROOT)


In [ ]:
from trajectory_tracing import TrajectoryTracer

trace_dir = PROJECT_ROOT / 'test_outputs' / 'series_tracing'
trace_dir.mkdir(parents=True, exist_ok=True)

tracer = TrajectoryTracer(
    topic='synthetic tracing run',
    provider='openai',
    model='gpt-4.1-mini',
    enabled=True,
    output_dir=trace_dir,
)

tracer.log_phase('planning', 'started')
tracer.log_message_snapshot('planner', 'step_1 use web_search')
tracer.log_tool_call('web_search', query='AI planning 2026')
tracer.log_tool_result('web_search', ok=True, latency_ms=123, result_preview='sample result')
tracer.log_phase('planning', 'completed')
tracer.complete(status='success', report_text='sample final report')

print('jsonl=', tracer.file_path)
print('summary=', tracer.summary_path)


In [ ]:
summary = json.loads(tracer.summary_path.read_text())
rows = [json.loads(line) for line in tracer.file_path.read_text().splitlines() if line.strip()]

required = {'run_started', 'phase', 'tool_call', 'tool_result', 'run_completed', 'final_report'}
seen = {r['event_type'] for r in rows}

print('event_count=', len(rows))
print('seen=', seen)
print('summary_status=', summary['status'])

assert required.issubset(seen)
assert summary['status'] == 'success'
print('Tracing validation passed.')
